In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Define image parameters
IMG_SIZE = 227 # AlexNet's input size
CHANNELS = 3


from google.colab import drive
drive.mount('/content/drive')

# --- Configuration ---
# *** IMPORTANT: UPDATE THIS PATH ***
DATA_DIR = '/content/drive/MyDrive/1roo6NzL3-VI-uWpVYut8aHBun5U36PJU'
MODEL_SAVE_PATH = '/content/drive/MyDrive/alexnet_student_id_model.h5'
ENCODER_SAVE_PATH = '/content/drive/MyDrive/label_encoder.pkl'
# ---------------------

images = []
labels = []
# Assuming the student ID folders are directly inside DATA_DIR
all_student_ids = os.listdir(DATA_DIR)

print(f"Found {len(all_student_ids)} student ID folders.")

# Loop through each student ID folder
for student_id in all_student_ids:
    student_dir = os.path.join(DATA_DIR, student_id)
    if not os.path.isdir(student_dir):
        continue

    for img_name in os.listdir(student_dir):
        img_path = os.path.join(student_dir, img_name)
        if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            images.append(img)
            labels.append(student_id)

        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            continue

# Convert lists to NumPy arrays
X = np.array(images, dtype="float32") / 255.0  # Normalize pixel values
y = np.array(labels)

print(f"Total images loaded: {len(X)}")


In [ ]:
# Encode student IDs into numerical classes
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
# Convert integers to one-hot encoding
y_encoded = to_categorical(integer_encoded)

NUM_CLASSES = len(label_encoder.classes_)
print(f"Number of classes (Student IDs): {NUM_CLASSES}")

# SAVE the LabelEncoder object
with open(ENCODER_SAVE_PATH, 'wb') as file:
    pickle.dump(label_encoder, file)
print(f"Label Encoder saved to: {ENCODER_SAVE_PATH}")


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"X_train shape: {X_train.shape}")



In [ ]:
def create_alexnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    # 1st Convolutional Block
    x = Conv2D(96, (11, 11), strides=(4, 4), activation='relu', padding='valid')(inputs)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)
    x = tf.nn.local_response_normalization(x)

    # 2nd Convolutional Block
    x = Conv2D(256, (5, 5), strides=(1, 1), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)
    x = tf.nn.local_response_normalization(x)

    # 3rd, 4th, 5th Convolutional Layers
    x = Conv2D(384, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
    x = Conv2D(384, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    # Fully Connected Block
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output Layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create and compile the model
alexnet_model = create_alexnet((IMG_SIZE, IMG_SIZE, CHANNELS), NUM_CLASSES)
alexnet_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

alexnet_model.summary()


In [ ]:

EPOCHS = 30
BATCH_SIZE = 32

print("Starting model training...")

history = alexnet_model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

# Save the trained model
alexnet_model.save(MODEL_SAVE_PATH)
print(f"\nModel saved to: {MODEL_SAVE_PATH}")